<a href="https://colab.research.google.com/github/Sarpedonic/Investigation-AutoML/blob/main/Model_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
#Model Search

In [ ]:
# Clone the repo
!git clone https://github.com/google/model_search.git

In [5]:
# Access folder
%cd model_search

/content/model_search


In [ ]:
# Install required packages
!pip install -r requirements.txt

In [7]:
# Protocol Buffers required
!protoc --python_out=./ model_search/proto/phoenix_spec.proto
!protoc --python_out=./ model_search/proto/hparam.proto
!protoc --python_out=./ model_search/proto/distillation_spec.proto
!protoc --python_out=./ model_search/proto/ensembling_spec.proto
!protoc --python_out=./ model_search/proto/transfer_learning_spec.proto

In [8]:
import pandas as pd

import model_search
from model_search import constants
from model_search import single_trainer
from model_search.data import csv_data

In [81]:
!ls

CONTRIBUTING.md  LICENSE       requirements.txt  WCDML.csv
food2.zip	 model_search  srep00196-s2.csv  WORKSPACE
food3.zip	 README.md     wcdml.csv


In [28]:
wcd_sets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dated_wcd_sets.csv")

In [42]:
from datetime import datetime as dt

In [43]:
meh = [dt.strptime(date, '%m/%d/%y').date().toordinal() for date in wcd_sets['Date']]

In [46]:
wcd_sets['Date'] = pd.Series(meh)

In [62]:
numeric = ['FiGMA_TDAC','FiGMA_TDA3','FiGMA_TDA5','Opp_FiGMA_TDAC','Opp_FiGMA_TDA3','Opp_FiGMA_TDA5']

In [ ]:
categorical = ['Team','Opp_Team','Set']

final_boss = pd.get_dummies(data=wcd_sets[categorical], columns=['Team','Opp_Team','Set'])

In [81]:
derts = pd.concat([wcd_sets['Set_Win'], final_boss, wcd_sets[numeric]], axis=1)

In [74]:
derts.shape

(17591, 290)

In [83]:
derts.to_csv('derts.csv')

In [90]:
## Use Model Search

In [78]:
# Single trainer
trainer = single_trainer.SingleTrainer(
                        data=csv_data.Provider(
                            label_index=0,
                            logits_dimension=2,
                            record_defaults=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                             0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                            filename="/content/drive/MyDrive/Colab Notebooks/wcd_ohe.csv"),
                        spec='/content/model_search/model_search/configs/dnn_config.pbtxt')


In [32]:
!mkdir /tmp/run_example/

mkdir: cannot create directory ‘/tmp/run_example/’: File exists


In [34]:
# This code is necessary to not get a flag parsing error
import sys
from absl import app

sys.argv = sys.argv[:1]

try:
  app.run(lambda argv: None)
except:
  pass

In [ ]:
trainer.try_models(
    number_models=200,
    train_steps=1000,
    eval_steps=100,
    root_dir="/tmp/run_example",
    batch_size=32,
    experiment_name="example2",
    experiment_owner="model_search_user")

In [ ]:
# Saved model to "/tmp/run_example/tuner-1/200/saved_model/temp-1615774350/saved_model.pb"
# Model Search performance
'accuracy': 0.6121875, 'auc_pr': 0.6077054, 'auc_roc': 0.6530719, 'loss': 0.6581666, 'num_parameters': 1315686